In [1]:
import bluesky
import bluesky.plan_stubs as bps
from apstools.synApps import SscanRecord
from ophyd import Device, EpicsSignal, EpicsSignalRO, Component, EpicsMotor
from apstools.plans import run_blocking_function
from ophyd.status import Status
import numpy as np
from epics import caput, caget
import logging
import time
import h5py
import os
import sys
import subprocess
import pathlib
os.environ.pop('QT_QPA_PLATFORM', None)
os.environ.pop('QT_QPA_PLATFORM_PLUGIN_PATH', None)
sys.path.append("/home/beams/STAFF19ID/bluesky/")
# sys.path.append(str(pathlib.Path(__file__).absolute().parent))
from instrument.utils.trajectories import *

/home/beams/STAFF19ID/bluesky/instrument/_iconfig.py


In [2]:
def mkdir(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
            print(f"Created directory: {directory}")
        except OSError as e:
            print(f"Failed to create directory: {directory} - {e}")
    else:
        print(f"Directory already exists: {directory}")

In [3]:
def mksubdirs(save_path, subdirs=[]):
    for folder in subdirs:
        path = f"{save_path}{folder}"
        mkdir(path)

In [4]:
def pvget(pv):
    try:
        cmd = f'pvget {pv}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e


In [5]:
def pvput(pv, value):
    try:
        cmd = f'pvput {pv} {value}'
        result = subprocess.getstatusoutput(cmd)
    except subprocess.CalledProcessError as e:
        result = e

In [6]:
def run_subprocess(command_list):
    try:
        result = subprocess.getstatusoutput(command_list)
    except subprocess.CalledProcessError as e:
        result = e
        pass
    return result 

In [7]:
def scan_number_in_list(lst, partial_str):
    matches = [s for s in lst if partial_str in s]
    if len(matches)>1:
        print("warning, more than one file matching scan number somehow")
    elif len(matches)==0:
        print("warning, no matches found, check that files are being saved and closing correctly")
    elif len(matches) == 1:
        pass
    else: 
        print("not sure how we got here, something very wrong")
    return matches[0]

In [8]:
def setup_scanrecord(scan1, scan2, scan_type, m1_name, m2_name, xarr, yarr, dwell_time, loop1_npts, trigger1="", trigger2="", trigger3="", trigger4="",):
    
    print("in setup_scan function")
    scan1.wait_for_connection()
    scan2.wait_for_connection()
    # yield from bps.mv(scaler1.preset_time, ct)  # counting time/point
    yield from run_blocking_function(scan1.reset)
    yield from run_blocking_function(scan2.reset)
    yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

    caput(f"{scan1.prefix}.P1PA", list(xarr))
    caput(f"{scan2.prefix}.P1PA", list(yarr))

    if scan_type=="fly":
        yield from bps.mv(scan1.positioner_delay, 0) #if fly scanning, positioner move at speed=step_size/dwell_time, no delay needed
    else: 
        yield from bps.mv(scan1.positioner_delay, dwell_time) #if step scanning, positioners move at 'fast' speed and dwell for specified dwell time at each position.
    
    # positioners
    yield from bps.mv(
        scan1.positioners.p1.mode, 1,
        scan1.positioners.p1.readback_pv, f"{m1_name}.RBV",
        scan1.positioners.p1.setpoint_pv, f"{m1_name}",
        scan1.number_points, len(xarr),
        scan2.positioners.p1.mode, 1,
        scan2.positioners.p1.readback_pv, f"{m2_name}.RBV",
        scan2.positioners.p1.setpoint_pv, f"{m2_name}",
        scan2.number_points, len(yarr),
        scan2.triggers.t1.trigger_pv, trigger1,
        scan2.triggers.t2.trigger_pv, trigger2,
        scan2.triggers.t3.trigger_pv, trigger3,
        scan2.triggers.t4.trigger_pv, trigger4,
    )


In [9]:
class SaveData(Device):
    file_system = Component(EpicsSignal, 'fileSystem', string=True)
    subdirectory = Component(EpicsSignal, 'subDir', string=True)
    base_name = Component(EpicsSignal, 'baseName', string=True) #basename needs _ at teh end
    scanNumber = Component(EpicsSignal, 'scanNumber')

    def setup_savedata(savedata, file_system, base_name, reset_counter=False):
        print("in setup_savedata function")
        # savedata.wait_for_connection()
        # yield from bps.mv(scaler1.preset_time, ct)  # counting time/point
        # yield from run_blocking_function(savedata.reset)
        yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

        subdir = base_name
        if base_name[-1] == "_":
            subdir = base_name.strip("_")
            
        if base_name[-1] != "_":
            base_name = base_name+"_"
    
        if reset_counter:
            yield from bps.mv(
            savedata.next_scan_number, 0)
            
        caput(savedata.file_system.pvname, str(file_system))
        yield from bps.mv(
            # savedata.file_system, file_system,
            savedata.subdirectory, f"{subdir}/mda/",
            savedata.base_name, f"/{base_name}",  
        )


In [21]:
class Xspress3(Device):
    ERASE = Component(EpicsSignal, 'det1:ERASE')
    Acquire = Component(EpicsSignal, 'det1:Acquire')
    AcquireTime = Component(EpicsSignal, 'det1:AcquireTime')
    NumImages = Component(EpicsSignal, 'det1:NumImages')
    ArrayCounter_RBV = Component(EpicsSignal, 'det1:ArrayCounter_RBV')
    EraseOnStart = Component(EpicsSignal, 'det1:EraseOnStart')
    DetectorState_RBV = Component(EpicsSignal, 'det1:DetectorState_RBV')
    TriggerMode = Component(EpicsSignal, 'det1:TriggerMode')
    EnableCallbacks = Component(EpicsSignal, 'Pva1:EnableCallbacks')
    Capture = Component(EpicsSignal, 'HDF1:Capture')
    FilePath = Component(EpicsSignal, 'HDF1:FilePath', string=True)
    FileName = Component(EpicsSignal, 'HDF1:FileName', string=True)
    FileNumber = Component(EpicsSignal, 'HDF1:FileNumber')
    FileWriteMode = Component(EpicsSignal, 'HDF1:FileWriteMode')
    FileTemplate = Component(EpicsSignal, 'HDF1:FileTemplate', string=True)
    AutoIncrement = Component(EpicsSignal, 'HDF1:AutoIncrement')
    AutoSave = Component(EpicsSignal, 'HDF1:AutoSave')
    NumCaptured_RBV = Component(EpicsSignal, 'HDF1:NumCaptured_RBV')

    def setup_xspress3(xp3, npts, sample_name, save_path, dwell_time, trigger_mode, scanNumber, reset_counter=False):
        print("in setup_xspress3 function")
        xp3.wait_for_connection()
        # yield from run_blocking_function(pm1.abort) # TODO: re-implement reset function for profile move
        yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

        caput(xp3.Capture.pvname, 0)
        caput(xp3.Acquire.pvname, 0)
        if reset_counter: 
            yield from bps.mv(xp3.FileNumber, 0)

        yield from bps.mv(
            xp3.ERASE, 1,
            xp3.NumImages, npts,
            xp3.AcquireTime, dwell_time,
            xp3.EraseOnStart, 0,
            xp3.TriggerMode, trigger_mode, 
            xp3.EnableCallbacks, 1, 
            xp3.AutoIncrement, 1,
            # xp3.FileNumber, scanNumber,
            xp3.AutoSave, 1,
            xp3.FileWriteMode, 2,
            xp3.FilePath, save_path,
            xp3.FileName, sample_name,
            xp3.FileTemplate, f"%s%s_%05d.h5",
            xp3.Capture, 1,
            xp3.Acquire, 1,
            )


In [11]:
class TetraMM(Device):
    Acquire = Component(EpicsSignal, 'Acquire')
    AcquireMode = Component(EpicsSignal, 'AcquireMode')
    Range = Component(EpicsSignal, 'Range')
    ValuesPerRead = Component(EpicsSignal, 'ValuesPerRead')
    AveragingTime = Component(EpicsSignal, 'AveragingTime')
    FastAveragingTime = Component(EpicsSignal, 'FastAveragingTime')
    FastAverageScan_scan = Component(EpicsSignal, 'FastAverageScan.SCAN')
    EmptyFreeList = Component(EpicsSignal, 'EmptyFreeList')
    TriggerMode = Component(EpicsSignal, 'TriggerMode')
    NumAcquire = Component(EpicsSignal, 'NumAcquire')
    Capture = Component(EpicsSignal, 'netCDF1:Capture')
    FilePath = Component(EpicsSignal, 'netCDF1:FilePath', string=True)
    FileName = Component(EpicsSignal, 'netCDF1:FileName', string=True)
    NumCapture = Component(EpicsSignal, 'netCDF1:NumCapture')
    FileNumber = Component(EpicsSignal, 'netCDF1:FileNumber')
    FileTemplate = Component(EpicsSignal, 'netCDF1:FileTemplate', string=True)
    AutoIncrement = Component(EpicsSignal, 'netCDF1:AutoIncrement')
    AutoSave = Component(EpicsSignal, 'netCDF1:AutoSave')
    WriteFile = Component(EpicsSignal, 'netCDF1:WriteFile')
    FileWriteMode = Component(EpicsSignal, 'netCDF1:FileWriteMode')
    WriteStatus = Component(EpicsSignal, 'netCDF1:WriteStatus')
    FilePathExists_RBV = Component(EpicsSignal, 'netCDF1:FilePathExists_RBV')
    
    def setup_tetramm(tmm, npts, sample_name, save_path, dwell_time, trigger_mode, scanNumber, reset_counter=False):
        print("in setup_tetramm function")
        tmm.wait_for_connection()
        # yield from run_blocking_function(pm1.abort) # TODO: re-implement reset function for profile move
        yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.
        caput(tmm.Capture.pvname, 0)
        caput(tmm.Acquire.pvname, 0)
        
        if reset_counter: 
            yield from bps.mv(tmm.FileNumber, 0)

        yield from bps.mv(
            tmm.TriggerMode, trigger_mode,
            tmm.NumAcquire, npts,
            tmm.NumCapture, npts, 
            tmm.EmptyFreeList, 1,
            tmm.ValuesPerRead, int(100000*dwell_time - 1), #100,000/dwell_time(ms)
            tmm.AveragingTime, dwell_time, 
            tmm.FastAveragingTime, dwell_time,
            tmm.AutoIncrement, 1,
            tmm.fileNumber, scanNumber,
            tmm.AutoSave, 1,
            tmm.FileWriteMode, 2,
            tmm.FilePath, save_path,
            tmm.FileName, sample_name,
            tmm.FileTemplate, f"%s%s_%05d.nc",
            tmm.Capture, 1,
            tmm.Acquire, 1,
            )
        

In [12]:
class SoftGlueZinq(Device):
    npts = Component(EpicsSignal, 'SG:plsTrn-1_NPULSES')
    period = Component(EpicsSignal, 'SG:plsTrn-1_PERIOD')
    width = Component(EpicsSignal, 'SG:plsTrn-1_WIDTH')
    clr1_proc = Component(EpicsSignal, 'SG:UpDnCntr-1_CLEAR_Signal.PROC')
    clr2_proc = Component(EpicsSignal, 'SG:UpDnCntr-2_CLEAR_Signal.PROC')
    clrcntr_proc = Component(EpicsSignal, 'SG:UpCntr-1_CLEAR_Signal.PROC')
    clr_C = Component(EpicsSignal, '1acquireDma.C')
    clr_F = Component(EpicsSignal, '1acquireDma.F')
    clr_D = Component(EpicsSignal, '1acquireDma.D')
    VALF = Component(EpicsSignal, '1acquireDma.VALF')
    VALG = Component(EpicsSignal, '1acquireDma.VALG')
    VALC = Component(EpicsSignal, '1acquireDma.VALC')
    enbl_dma = Component(EpicsSignal, '1acquireDmaEnable')
    usrclk_enable = Component(EpicsSignal, 'SG:DivByN-3_ENABLE_Signal.PROC')
    rst_buffer = Component(EpicsSignal, 'SG:BUFFER-1_IN_Signal.PROC')
    send_pulses = Component(EpicsSignal, 'SG:plsTrn-1_Inp_Signal.PROC')
    def setup_softgluezinq(sgz, npts, period): 
        print("in setup_softgluezinq function")
        clk_f = 20E6 # master clock
        #TODO: fix period definition
        period = int(period*clk_f) #period in number of clock cycles 
        width = int(0.000001*clk_f) #1us in number of clock cycles
        
        caput(sgz.usrclk_enable.pvname, "0") #disable user clock 
        caput(sgz.enbl_dma.pvname, 0) #disable user clock 
            
        yield from bps.mv(
            sgz.npts, npts,       #set npts
            sgz.period, period,   #set period in # of clock cycles
            sgz.width, width,     #pulse width in # of clock cycles
            sgz.clr1_proc, 1,     #clear interferometer 1
            sgz.clr2_proc, 1,     #clear interfoerometer 2
            # sgz.clr_C, 1,         #clears SOMETHING
            # sgz.clr_F, 1,         #clears clears cbuff
            sgz.clr_D, 1,         #clears plots
            # sgz.rst_buffer, 1,    #resets buff (do i need this?)
            sgz.enbl_dma, 1,      #enables position collection
            )
        caput(sgz.usrclk_enable.pvname, "0") #enables user clock 


In [13]:
class PositionerStream(Device):
  reset_ = Component(EpicsSignal, 'reset')
  start_ = Component(EpicsSignal, 'start')
  stop_ = Component(EpicsSignal, 'stop')
  status = Component(EpicsSignal, 'status')
  outputFile = Component(EpicsSignal, 'outputFile')

  def setup_positionstream(filename, filepath):
    print("in setup_positionstream function")
    cmd = f"pvput -r \"filePath\" posvr:outputFile \'{{\"filePath\":\"{filepath}\"}}\'"
    print(run_subprocess(cmd)[1])
    cmd = f"pvput -r \"fileName\" posvr:outputFile \'{{\"fileName\":\"{filename}\"}}\'"
    print(run_subprocess(cmd)[1])


In [14]:
def create_master_file(basedir, sample_name, scan_number, groups=["flyXRF", "eiger", "mda", "tetramm", "positions"]):
    #TODO: samples with the same name get saved to the same folder even if separate scan, need to link files based on scan number instead of all files in folder. 
    with h5py.File(f"{basedir}/{sample_name}_{scan_number}_master.h5", "w") as f:
        for group in groups:
            f.create_group(group)
            files = os.listdir(f"{basedir}/{group}")
            if group == "tetramm":
                files = [file for file in files if file.split(".")[-1]=="nc"]
                file = scan_number_in_list(files, str(scan_number))
                string_data = [file.encode('utf-8')]
                dset = f[group].create_dataset("fnames", data=string_data)
            elif group == "mda":
                files = [file for file in files if file.split(".")[-1]=="mda"]
                file = scan_number_in_list(files, str(scan_number))
                string_data = [file.encode('utf-8')]
                dset = f[group].create_dataset("fnames", data=string_data)
            elif group == "flyXRF": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                file = scan_number_in_list(files, str(scan_number))
                f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/entry")
            elif group == "positions": 
                files = [file for file in files if file.split(".")[-1]=="h5"]
                file = scan_number_in_list(files, str(scan_number))
                f[f"/{group}/{file}"] = h5py.ExternalLink(f"/{group}/{file}", "/stream")

In [23]:
def run_scan(scan_type="fly", trajectory="snake", loop1="2idsft:m1", loop2="2idsft:m2", dwell_time=10, 
             devices={"flyXRF":"XSP3_1Chan:", "tetramm:":"2idsft:TetrAMM1", "scanrecord":["2idsft:scan1", "2idsft:scan2"], "softglue": "2idMZ1:", "positions":"posvr:"},
             sample_name="sample_name", pi_directory="/mnt/micdata1/save_dev/", comments="", reset_counter=False,
             l1_center=0, l1_size=0.01, l1_width=0.5, l2_center=0, l2_size=0.01, l2_width=0.5,
             ):
        
    """parse parameters""" 
    if trajectory == "snake":
        x, y, t = snake(dwell_time, l1_size, l1_center, l2_center, l1_width, l2_width)
    elif trajectory == "raster":
        x, y, npts_line, npts_tot = raster_sr(dwell_time, l1_size, l1_center, l2_center, l1_width, l2_width)
    elif trajectory == "spiral":
        pass
    elif trajectory == "lissajous":
        pass
    elif trajectory == "custom":
        pass
        
    x = list(np.round(x, 5))
    y = list(np.round(y, 5))
    dwell = dwell_time/1000    
    folder_name = sample_name.strip("_")
    save_path = f"{pi_directory}{folder_name}/"
    mkdir(save_path)
    subdirs = [detector for detector in devices if detector in ("flyXRF", "tetramm", "positions", "eiger", "xmap")]
    mksubdirs(save_path, subdirs)
    """Set up positioners (move to starting pos)"""
    #TODO: of the parameters in loop1-loop4, figure out which are motors somehow or hardcode them in the devices folder and then import them here. 

    """trigger source"""
    use_softglue_triggers = "softglue" in list(devices.keys())
    
    """setup devices"""

    if "softglue" in devices:
        sgz = SoftGlueZinq(devices["softglue"], name = "sgz")
        yield from SoftGlueZinq.setup_softgluezinq(sgz, npts_line, dwell)
        trigger2 = sgz.send_pulses.pvname
    else:
        trigger2=""
        
    if "scanrecord" in devices:
        mksubdirs(save_path, ["mda"])
        
        scan1 = SscanRecord(devices["scanrecord"][0], name="scan1")
        scan2 = SscanRecord(devices["scanrecord"][1], name="scan2")
        trigger1 = scan1.execute_scan.pvname
        yield from setup_scanrecord(scan1, scan2, scan_type, loop1, loop2, x, y, dwell, npts_line, trigger1=trigger1, trigger2=trigger2)
        prefix = scan1.prefix.split(":")[0]+":saveData_"
        savedata = SaveData(prefix, name="savedata")
        yield from SaveData.setup_savedata(savedata, pi_directory, sample_name, reset_counter=False)
        scanNumber = savedata.scanNumber.value
    else:
        print("scanrecord not specified, cannot scan")
        return
    
    if "flyXRF" in devices:
        if use_softglue_triggers: 
            trigger_mode = 3 #ext trigger
        else: 
            trigger_mode = 1 #internal
        xp3 = Xspress3(devices["flyXRF"], name="xp3")
        savepath = f"{save_path}flyXRF"
        yield from Xspress3.setup_xspress3(xp3, npts_tot, sample_name, savepath, dwell, trigger_mode, scanNumber, reset_counter=False)
    
    if "tetramm"in devices:
        if use_softglue_triggers: 
            trigger_mode = 1 #ext trigger
        else: 
            trigger_mode = 0 #internal
        tmm = TetraMM(devices["tetramm"], name="tmm")
        savepath = f"{save_path}tetramm"
        yield from TetraMM.setup_tetramm(tmm, npts_tot, sample_name, savepath, dwell, trigger_mode, scanNumber, reset_counter=False)
        
    if "positions" in devices:
        filenumber = "{:05d}".format(scanNumber)
        postrm = PositionerStream(devices["positions"], name="postrm")
        filename = f"positions_{filenumber}.h5"
        PositionerStream.setup_positionstream(filename, f"{save_path}positions") 
    else: 
        #TODO: figure out a way to increment softglue filenumber whvery time it closes and a way to reset counter. 
        print("file number not tracked. Not sure how else to set file name if not based on another detector's filenumber")

    """setup motors"""
    m1 = EpicsMotor(loop1, name="m1")
    m2 = EpicsMotor(loop2, name="m2")
    if scan_type == "fly":
        #TODO: add component to epics motor to get maximum velocity and acceleration
        #TODO: add and setup additional motors if other loops are motors.. somehow.
        #set motor velocity = sep_size/dwell_time
        yield from bps.mv(m1.velocity, 3, m1.acceleration, 0.1, m2.velocity, 3, m2.acceleration, 0.1)
        m1.move(x[0], wait=True)
        m2.move(y[0], wait=True)
        vel = l1_size/dwell #vel is in mm/s
        yield from bps.mv(m1.velocity, vel)
        
    else: 
        yield from bps.mv(m1.velocity, 3, m1.acceleration, 0.1, m2.velocity, 3, m2.acceleration, 0.1)
        m1.move(x[0], wait=True)
        m2.move(y[0], wait=True)
    
    """Start executing scan"""
    print("Done setting up scan, about to start scan")
    st = Status()
    def watch_execute_scan(old_value, value, **kwargs):
        # Watch for scan1.EXSC to change from 1 to 0 (when the scan ends).
        if old_value == 1 and value == 0:
            # mark as finished (successfully).
            st.set_finished()
            # Remove the subscription.
            scan2.execute_scan.clear_sub(watch_execute_scan)

    # TODO need some way to check if devices are ready before proceeding. timeout and exit with a warning if something is missing. 
    # if motors.inpos and pm1.isready and tmm.isready and xp3.isready and sgz.isready and postrm.isready: 
    time.sleep(2)
    ready = True 
    if ready: 
        print("executing scan")
        # caput(tmm.Acquire.pvname, 1) #begin acquiring tetramm
        # caput(xp3.Acquire.pvname, 1) #begin acquiring xspress3
        pvput(postrm.start_.pvname, 1) #begin position stream
        
    yield from bps.mv(scan2.execute_scan, 1)
    scan2.execute_scan.subscribe(watch_execute_scan)
    yield from run_blocking_function(st.wait)
    pvput(postrm.stop_.pvname, 1) #stop position stream
    
    """Set up masterFile"""
    create_master_file(save_path, sample_name, scanNumber, subdirs)
    
    print("end of plan\n")


In [24]:
RE = bluesky.RunEngine() 
RE(run_scan(scan_type="fly", trajectory="raster", loop1="2idsft:m1", loop2="2idsft:m2", dwell_time=10, 
            devices={"flyXRF":"XSP3_1Chan:", "scanrecord":["2idsft:scan1", "2idsft:scan2"], "softglue": "2idMZ1:", 
                     "positions":"posvr:"}, sample_name="sample_name", pi_directory="/net/s19data/export/ISN/save_dev/", 
            comments="", reset_counter=False, l1_center=0, l1_size=0.01, l1_width=0.2, l2_center=0, l2_size=0.01, l2_width=0.2, ))

#"tetramm":"2idsft:TetrAMM1:"

Directory already exists: /net/s19data/export/ISN/save_dev/sample_name/
Directory already exists: /net/s19data/export/ISN/save_dev/sample_name/flyXRF
Directory already exists: /net/s19data/export/ISN/save_dev/sample_name/positions
in setup_softgluezinq function
Directory already exists: /net/s19data/export/ISN/save_dev/sample_name/mda
in setup_scan function
in setup_savedata function
in setup_xspress3 function
in setup_positionstream function
Old : structure 
    string filePath /net/s19data/export/ISN/save_dev/sample_name/positions/
New : structure 
    string filePath /net/s19data/export/ISN/save_dev/sample_name/positions/
Old : structure 
    string fileName positions_00005.h5
New : structure 
    string fileName positions_00006.h5
Done setting up scan, about to start scan
executing scan
warning, more than one file matching scan number somehow
warning, more than one file matching scan number somehow
end of plan



()